In [0]:
!pip install tensorflow==1.14
%tensorflow_version 1.x

In [0]:
import numpy as np
import matplotlib.pyplot as plt
from keras.datasets import mnist
import tensorflow as tf
import keras
from tqdm import tqdm

In [0]:
def preprocess_img(x):
    """
    The function to return the preprocessed images. 
    You dont need to preprocess for sure, but use this if you are!!
    
    Suggest some different ways to preprocess images!!!
    
    YOUR CODE GOES HERE
    """

    # Normalizing the image for easier weight calculations

    x = x.astype('float32')
    x /= 255
    return x

In [0]:
def import_mnist(preprocess=True):
    
    print("Downloading MNIST data ... ", end = "")

    # downloads the data from the mnist dataset incase it is unavailble in your system - one time operation only.
    from keras.datasets import mnist
    
    # loading data from the MNIST DIGIT dataset. We will be using only x_train as the model just needs to generate images 
    # which are real and indistinguishable. Hence we are not worried about which digit the model is outputing.
    
    (x_train, y_train), (x_test, y_test) = mnist.load_data()
    # reshaping the input to pass to keras - input shape = (number of images, image_dimensions, 1 - number of channels) : Black and white image hence single channel.
    x_train = x_train.reshape(x_train.shape[0], 28,28,1)
    x_test = x_test.reshape(x_test.shape[0], 28,28,1)
    
    """
    Preprocessing the images read from the dataset.
    if preprocess:
        x_train = preprocess_img(x_train)
        x_test = preprocess_img(x_test)
    
    You dont need to do this for sure but still think about it!
    """
    
    """
    YOUR CODE GOES HERE!
    """
    if(preprocess):
      x_train = preprocess_img(x_train)
      x_test = preprocess_img(x_test)

    print("Done")
    return x_train, y_train, x_test, y_test


In [0]:
"""
Return a numpy array containing size number of randomly generated numbers each of length 100. HINT : use numpy random functions.
"""
def gen_noise(size):
    """
    YOUR CODE GOES HERE!!
    """

    return np.random.uniform(-1.0, 1.0, size=[size, 100])


In [0]:
"""
Like i said everytime the entire GAN is trained the discriminator's weights are fixed. Hence this function is used to do that
Given a network net it converts all its weights either to Frozen (fixed) - if False and back to trainable mode if True
"""
def make_trainable(net, val):
    net.trainable = val
    for l in net.layers:
        l.trainable = val

In [0]:
"""
Plotting helper function - I did it for you, so take LITE!!!
"""
        
def plot_large(img):
    fig1 = plt.figure(figsize=(4,4))
    ax1 = fig1.add_subplot(1,1,1)
    ax1.axes.get_xaxis().set_visible(False)
    ax1.axes.get_yaxis().set_visible(False)
    ax1.imshow(img, cmap="gray")
    plt.show()
def plot(generator,n=5):
    img = np.zeros((n*28,1))
    for i in range(n):
        k = generator.predict(gen_noise(n)).reshape(n*28,28)
        img = np.concatenate((img,k), axis=1)
    plot_large(img[:,1:])

In [0]:
class DCGAN():

    # the init function which is called automatically when we initialize an object of the class
    def __init__(self, learning_rate, batch_size, num_epochs, save_path):
        # the input, output specific parameters!!
        self.img_shape = (28,28,1)
        self.noise_dim = 100

        # model specific parameters!!
        self.lr = learning_rate
        self.batch_size = batch_size
        self.num_epochs = num_epochs
        self.save_path = save_path # where we save the models weights in a regular basis.

        # defining the generator and discriminator.
        """
        Use the helper functions in this class to define the generator and discriminator!!!
        Use a sequential model to define the gan_model using keras.Sequential to connect the generator to the discriminator.

        YOUR CODE GOES HERE .......

        """

        self.discriminator = self.create_disc()
        self.generator = self.create_gen()

        self.gan_model = keras.Sequential()
        self.gan_model.add(self.generator)
        self.gan_model.add(self.discriminator)

        print("GAN Model : ")
        print(self.gan_model.summary())
        # compile the models - I have done this for you .....................
        """
        BONUS : what is binary cross entropy and why is it used here. I am asking very simple stuff :P (easy bonus)
        """

        """
        BONUS ANSWER :
        
        Binary Cross-Entropy is the loss function usually associated with binary output labels.
        
        It is used here, since the output is binary (Real/Fake)
        """
        self.discriminator.compile(keras.optimizers.Adam(2*self.lr), "binary_crossentropy")
        self.gan_model.compile(keras.optimizers.Adam(self.lr), "binary_crossentropy")

    """
    create_gen function to define the generator model.
    Use the keras Sequential models to define the generator. Visit keras documentation page for more info!!

    For example:
    model = keras.Sequential()
    layer1 = keras.layers.Dense(n, activation="relu")
    model.add(layer1)
    return model
    """
    def create_gen(self):
        """
        model = keras.Sequential()
        layer1 = ...
        layer2 = .....
        layer3 = .....
        ..


        model.add(layer1)
        model.add(layer2)
        ....


        return model


        YOUR CODE GOES HERE!!!!
        """

        model = keras.Sequential()

        model.add(keras.layers.Dense(256*7*7, input_shape = (self.noise_dim,)))
        model.add(keras.layers.BatchNormalization())
        model.add(keras.layers.LeakyReLU(0.2))
        model.add(keras.layers.Reshape((7,7,256)))
        model.add(keras.layers.Dropout(0.4))

        model.add(keras.layers.UpSampling2D())
        model.add(keras.layers.Conv2DTranspose(128, kernel_size = 5, strides = 1, padding='same'))
        model.add(keras.layers.BatchNormalization(momentum=0.9))
        model.add(keras.layers.LeakyReLU(alpha=0.01))

        model.add(keras.layers.UpSampling2D())
        model.add(keras.layers.Conv2DTranspose(64, kernel_size=5, strides=1, padding='same'))
        model.add(keras.layers.BatchNormalization(momentum=0.9))
        model.add(keras.layers.LeakyReLU(alpha=0.2))

        model.add(keras.layers.Conv2DTranspose(32, kernel_size=5, strides=1, padding='same'))
        model.add(keras.layers.BatchNormalization(momentum=0.9))
        model.add(keras.layers.LeakyReLU(alpha=0.2))

        model.add(keras.layers.Conv2DTranspose(1, kernel_size = 5, strides = 1, padding='same'))
        model.add(keras.layers.Activation('sigmoid'))

        print("Generator model = ")
        print(model.summary())
        return model

    """
    create_disc function is used to create the discriminator for the model.
    Use the same method as above to define the model.

    """

    def create_disc(self):
        """
        model = keras.Sequential()
        layer1 = ...
        layer2 = .....
        layer3 = .....
        ..


        model.add(layer1)
        model.add(layer2)
        ....


        return model


        YOUR CODE GOES HERE!!!!
        """

        model = keras.Sequential()
        model.add(keras.layers.Conv2D(64,kernel_size=5,strides=2,padding='same',input_shape=self.img_shape))
        model.add(keras.layers.BatchNormalization(momentum=0.9))
        model.add(keras.layers.ReLU())
        model.add(keras.layers.Dropout(0.4))
        model.add(keras.layers.Conv2D(128,kernel_size=5,strides=2,padding='same'))
        model.add(keras.layers.BatchNormalization(momentum=0.9))
        model.add(keras.layers.ReLU())
        model.add(keras.layers.Dropout(0.4))
        model.add(keras.layers.Conv2D(256,kernel_size=5,strides=2,padding='same'))
        model.add(keras.layers.BatchNormalization(momentum=0.9))
        model.add(keras.layers.ReLU())
        model.add(keras.layers.Dropout(0.4))
        model.add(keras.layers.Conv2D(512,kernel_size=5,strides=1,padding='same'))
        model.add(keras.layers.BatchNormalization(momentum=0.9))
        model.add(keras.layers.ReLU())
        model.add(keras.layers.Dropout(0.4))
        model.add(keras.layers.Flatten())
        model.add(keras.layers.Dense(1))
        model.add(keras.layers.Activation('sigmoid'))

        print("Discriminator model = ")
        print(model.summary())
        return model

    # function to pretrain the discriminator model to identify real and fake images!!!
    def pretrain(self, x):
        # how many examples to choose
        size = x.shape[0]//200

        """
        randomly choose size number of images from x - real images
        Similarly create size number of fake images - how will you do this ! (brain time :P)

        Now decide on a convention - Lets say - 0 - Fake images, 1 - Real images

        For example:
        real_img = ...
        fake_img = ...
        x = np.concatenate(real_img, fake_img)
        labels = [1]*size + [0]*size - x contains first size number of real images (label 1) and next size number of fake images (label 0)
        self.discriminator.fit(x, labels, batch_size = self.batch_size, epochs = 1)



        YOUR CODE GOES HERE!!
        """

        x,y = self.gen_data(x,size)

        self.discriminator.fit(x,y,batch_size = self.batch_size,epochs = 1)

        print("\n\nPretraining done!\n\n")


    # function to generate data for the discriminator to train on in every iteration!
    def gen_data(self, x, size):
        """
        randomly choose size number of real images from x - real images
        also create size number of fake images - think think !!!!!
        concatenate them and create labels for the same and hence return them


        return x, labels

        YOUR CODE GOES HERE!!!

        """

        if(size>len(x)):
          raise ValueError("Size requested greater than size of x")
        
        real = x[np.random.choice(range(6000),size,replace=False)]
        fake = self.generator.predict(gen_noise(size))
        images = np.concatenate((real,fake))
        labels = np.concatenate((np.ones(size),np.zeros(size)))
        return np.array(images),np.array(labels)


    # the training function. Loop through every epoch and perform multiple iterations in each epoch with each iteration for a batch-size number of images!!
    def train(self, x, num_iter):
        # iterating through num_epochs!!

        for i in range(self.num_epochs):
            print("\n\nEpoch no :"+str(i+1)+"/"+str(self.num_epochs)+"\n\n")


            # iterating through num of iterations!!!
            for j in tqdm(range(num_iter)):
                # use the gen_data function to create input and output for the discriminator!!
                x1,y = self.gen_data(x, self.batch_size//2)

                # train the discriminator on this data!!!
                loss = self.discriminator.train_on_batch(x1,y)
                
                print("\nLoss in discriminator : "+str(loss),end="\n\n")

                # Freeze the discriminator to train the GAN model - fix the weights!!
                # make_trainable(self.discriminator, False)

                # train the gan model!!!!!!!!

                loss = self.gan_model.train_on_batch(np.random.uniform(-1.0, 1.0, size=[self.batch_size, 100]),np.ones([self.batch_size, 1]))


                print("\nLoss in gan : "+str(loss),end="\n\n")

                # make the discriminator params back to trainable for the next iteration!!!
                # make_trainable(self.discriminator, True)

            #save the weights and plot the results every 10 epochs!!!
            if i%10 == 0:
              self.gan_model.save_weights(self.save_path + str(i+1)+".h5")
            plot(self.generator)


In [0]:
def main(lr, batch_size, num_epochs, save_path):
    
    # Loading the dataset using the function import_mnist present in utils.py
    x,_,_,_ = import_mnist(preprocess=True)
    
    # defining an object of the Class DCGAN which is going to be our gan model. This function calls the init function of the class DCGAN
    gan_model = DCGAN(lr, batch_size, num_epochs, save_path)
    
    """
    YOUR CODE GOES HERE!!!......................... (use functions defined in the DCGAN CLASS)
    
    Start training the model.
    """

    # gan_model.pretrain(x)

    gan_model.train(x,1)
    
    print("Training Done!")

In [0]:
lr = 1e-4
batch_size = 128
num_epochs = 100
save_path = "./weights/"

main(lr, batch_size, num_epochs, save_path)